In [ ]:
### Importing packages


import nltk
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


### Input the window size required between keyword and commodity.Keep it less than 5 for better context. 

n = int(input("Enter required distance: "))
# Input the radius around commodity required to be extracted from the feedback message.
r = int(input("Enter required radius for feedback : "))
while(r<=n):
    print("Inappropriate radius")
    r = int(input("Enter the radius again: "))
 

### Importing the Data


xls = pd.ExcelFile(input("enter the path of excel file from where data is to be read"))
df1 = pd.read_excel(xls, 'Sheet1')
df2 = pd.read_excel(xls, 'Sheet2')
df3 = pd.read_excel(xls, 'Sheet3')


###  Data pre-processing

df3['Keywords']=df3['Keywords'].str.replace(r'\W',"")

#creating list and set for columns in excel file
messg=list(df1['Message']) 
Comp=set(df2['Components'])
key=set(df3['Keywords'])



# empty list to store back the splitted sentence
Mesg=[]
# empty list to store the commodities 
Commodity=[]
# empty list to store the keyword associated with respective commodity
Keyword=[]
# empty list to join words into sentences
combine = []


###  Finding context between component and keyword in every message 

for k in range(0,len(messg)) :


    # this is the list of tokens containing numbers and punctuations
    tokens = word_tokenize(messg[k])
    #tokens = [w.lower() for w in tokens]
    words = [word for word in tokens if word.isalpha()]
   
    
    #for storing indices of the matched component and keyword 
    ic = -1
    ik = -1
    
    # for keeping track of each word's index
    s = 0
    
    #for traversing every word of kth message
    for m in range(0,len(words)):
        
        #for matching every word with component set
        if(words[m] in Comp) :
            ic = s
            temp1 = ic
            Component=words[m]
            

        #for matching every word with keyword set    
        if(words[m] in key) :
            ik = s
            remark=words[m]
        #check if both matched then exit the loop    
        if((ic != -1) & (ik != -1)):
            #to ensure that ic is always greater than ik
            if(ic>ik):
                ic,ik = ik,ic
            #print(ik,ic)
        
            #calculating the distance between component and keyword
            d=(ik-ic-1) 

            #checking if distance is less than required distance, then extracting the feedback 
            if (d<=n):
                Commodity.append(Component)
                Keyword.append(remark)

                # extracting the feedback from the sentence
                if((temp1-r)<0):
                    str_m=words[0:temp1+r+1]
                elif(((temp1+r+1)>=len(words))):
                    str_m=words[temp1-r:]
                elif(((temp1-r)<0) & ((temp1+r+1)>=len(words))):
                    str_m=words[0:]
                else:
                    str_m=words[temp1-r:temp1+r+1]


                
                # joining the words into sentence
                combine =' '.join(str_m)
                combine =''.join(combine)
                #print(combine)
                
                # appending the sentences
                Mesg.append(combine)
                
            #for second word iteration    
            ic = -1 
            ik = -1



        # updating when a word is traversed to hold index of next word
        s=s+1
        


### Result

#forming list of messages and corresponding commodities and keywords
Result = {'Feedback': Mesg,'Commodity': Commodity, 'Keyword': Keyword}
#print(Result)



### Exporting the summarized feedback along with contextual commodity and keyword to excel file 


df = pd.DataFrame(Result, columns=['Feedback','Commodity', 'Keyword'])
df.to_excel(input("enter the path of excel file from where feedback is to be stored"))
print (df)
x=input()

